In [48]:
import pandas as pd
import requests
import ast
import mariadb
from logger.Logger import etlLogger
import sys
from datetime import datetime, timedelta
from modulos.conecta_db import connect_db, ingest_data
from modulos.crawler import get_response
from modulos.data_processing import create_df, cities_and_itens, get_urls

ImportError: cannot import name 'ingest_data' from 'modulos.conecta_db' (c:\Users\victo.000\Documents\python\albion_prices\modulos\conecta_db.py)

In [2]:
dt_today = datetime.now()
dt_tomorrow = datetime.now() + timedelta(days=1)
str_date_today = dt_today.strftime("%m-%d-%Y")
str_date_tomorrow = dt_tomorrow.strftime("%m-%d-%Y")

In [3]:
URL = f"https://www.albion-online-data.com/api/v2/stats/gold?date={str_date_today}&end_date={str_date_tomorrow}"
URL

'https://www.albion-online-data.com/api/v2/stats/gold?date=08-24-2022&end_date=08-25-2022'

In [4]:
try:
    conn = mariadb.connect(
        user="ciurlini",
        password="casa2406",
        host="localhost",
        port=3306,
        database="ALBION"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

In [5]:
response = requests.get(URL)
content = response.content
content_string = content.decode('UTF-8')
content_list = ast.literal_eval(content_string)

In [6]:
df_index = pd.DataFrame(content_list)
df_index = df_index[['timestamp', 'price']]
df_index[['timestamp']] = df_index[['timestamp']].apply(pd.to_datetime)

In [7]:
df_dbmaria = pd.read_sql(
    "SELECT  * FROM ALBION.gold_prices", conn)

c:\Users\victo.000\Documents\python\albion_prices\.pyenv\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [8]:
df_diff = pd.concat([df_dbmaria,df_index]).drop_duplicates(keep=False)

In [9]:
df_diff

,timestamp,price


In [10]:
df_ingest = df_diff.astype({'timestamp': str, 'price': int})

In [11]:
cols = "`,`".join([str(i) for i in df_ingest.columns.tolist()])

In [12]:
for i,row in df_ingest.iterrows():
    sql = "INSERT INTO `gold_prices` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cur.execute(sql, tuple(row))
conn.commit()

In [13]:
len(df_ingest)

0

In [14]:
list_of_cities = ['Bridgewatch', 'Caerleon', 'Lymhurst', 'Martlock', 'Thetford']
with open('config/list_of_itens.txt') as f:
    list_of_itens = [line.strip() for line in f.readlines()]
cities = ','.join(list_of_cities)
itens = ','.join(list_of_itens)

In [55]:
URL_ITEMS = f"https://www.albion-online-data.com/api/v2/stats/history/{itens}?date=02-02-2022&end_date={str_date_tomorrow}&locations={cities}&qualities=1,2,3&time-scale=6"

In [17]:
LOGGER_OBJ = etlLogger(project_name='item_predict')

In [56]:
content_list = get_response(URL_ITEMS, LOGGER_OBJ)

{"timestamp": "2022-08-24 20:35:35,781", "processo": "MainProcess", "thread": "MainThread","level": "INFO", "script": "crawler.py", "modulo": "crawler", "metodo": "get_response",  "mensagem": "Realizando requisição: https://www.albion-online-data.com/api/v2/stats/history/T4_POTION_HEAL,T6_POTION_HEAL,T7_POTION_STONESKIN,T4_POTION_COOLDOWN,T6_POTION_COOLDOWN,T4_POTION_HEAL@1,T6_POTION_HEAL@1,T7_POTION_STONESKIN@1,T4_POTION_COOLDOWN@1,T6_POTION_COOLDOWN@1?date=02-02-2022&end_date=08-25-2022&locations=Bridgewatch,Caerleon,Lymhurst,Martlock,Thetford&qualities=1,2,3&time-scale=6"}


In [61]:
len(content_list[1]['data'])

705

In [62]:
data_from_item = []
for all_info in content_list:
    data = all_info['data']
    for item_info in data:
        row_of_item = {'timestamp': item_info['timestamp'],
               'item_id': all_info['item_id'],
               'location': all_info['location'],
               'item_count': item_info['item_count'],
               'avg_price': item_info['avg_price']}
    data_from_item.append(row_of_item)

In [63]:
data_from_item = []
for all_info in content_list:
    data = all_info['data']
    for item_info in data:
        row_of_item = {'timestamp': item_info['timestamp'],
        'item_id': all_info['item_id'],
        'location': all_info['location'],
        'item_count': item_info['item_count'],
        'price': item_info['avg_price']}
        data_from_item.append(row_of_item)

In [64]:
df_ingest = pd.DataFrame(data_from_item)

In [65]:
df_ingest

,timestamp,item_id,location,item_count,price
0,2022-02-02T00:00:00,T4_POTION_COOLDOWN,Bridgewatch,16002,752
1,2022-02-02T06:00:00,T4_POTION_COOLDOWN,Bridgewatch,18884,1613
2,2022-02-02T12:00:00,T4_POTION_COOLDOWN,Bridgewatch,31106,701
3,2022-02-02T18:00:00,T4_POTION_COOLDOWN,Bridgewatch,24371,712
4,2022-02-03T00:00:00,T4_POTION_COOLDOWN,Bridgewatch,15241,708
...,...,...,...,...,...
34969,2022-08-13T06:00:00,T7_POTION_STONESKIN@1,Thetford,107,59265
34970,2022-08-13T12:00:00,T7_POTION_STONESKIN@1,Thetford,244,56461
34971,2022-08-13T18:00:00,T7_POTION_STONESKIN@1,Thetford,184,54002
34972,2022-08-14T00:00:00,T7_POTION_STONESKIN@1,Thetford,61,53204


In [66]:
df_ingest[['timestamp']] = df_ingest[['timestamp']].apply(pd.to_datetime)

In [67]:
df_ingest = df_ingest.astype({'timestamp': str, 'item_id': str, 'location': str, 'item_count': int, 'price': int})

In [69]:
cols = "`,`".join([str(i) for i in df_ingest.columns.tolist()])

try:
    for i,row in df_ingest.iterrows():
        sql = "INSERT INTO `potion_prices` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cur.execute(sql, tuple(row))
    conn.commit()
    LOGGER_OBJ.info("Ingestão realizada com sucesso")
except Exception as e:
    LOGGER_OBJ.error(f"Falha ao realizar ingestão de dados: {e}")

{"timestamp": "2022-08-24 20:41:49,292", "processo": "MainProcess", "thread": "MainThread","level": "INFO", "script": "485929014.py", "modulo": "485929014", "metodo": "<cell line: 3>",  "mensagem": "Ingestão realizada com sucesso"}
